In [1208]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

from agoda_cancellation_estimator import AgodaCancellationEstimator
from agoda_cancellation_prediction import load_data
import pandas as pd
import numpy as np


In [1209]:
full_data = pd.read_csv(r"C:\Users\97250\PycharmProjects\IML.HUJI\datasets\agoda_cancellation_train.csv").fillna(False)
test_data_w1 = pd.read_csv(r"C:\Users\97250\PycharmProjects\IML.HUJI\datasets\test_set_week_1.csv").dropna().drop_duplicates()
full_data.head()

,h_booking_id,booking_datetime,checkin_date,checkout_date,hotel_id,hotel_country_code,hotel_live_date,hotel_star_rating,accommadation_type_name,charge_option,...,request_highfloor,request_largebed,request_twinbeds,request_airport,request_earlycheckin,cancellation_datetime,hotel_area_code,hotel_brand_code,hotel_chain_code,hotel_city_code
0,-9223194055642672935,2018-06-28 21:15:00,2018-07-09 00:00:00,2018-07-13 00:00:00,6452,HK,2009-06-28 02:02:00,4.0,Hotel,Pay Now,...,False,False,False,False,False,False,1192,False,False,142
1,-9222713784330706132,2018-08-10 22:31:00,2018-08-16 00:00:00,2018-08-17 00:00:00,47729,CN,2011-06-07 11:52:00,4.0,Hotel,Pay Later,...,0.0,0.0,1.0,0.0,0.0,False,5617,902.0,600.0,1273
2,-9222411208325704942,2018-09-14 07:55:00,2018-09-14 00:00:00,2018-09-15 00:00:00,780431,KR,2014-11-20 15:43:00,4.0,Hotel,Pay Now,...,0.0,0.0,0.0,0.0,0.0,False,3819,False,False,2799
3,-9222220845872895471,2018-06-25 07:33:00,2018-07-02 00:00:00,2018-07-03 00:00:00,291365,JP,2011-11-21 12:27:00,3.0,Hotel,Pay Later,...,0.0,1.0,0.0,0.0,0.0,False,2379,466.0,236.0,1448
4,-9221127186162682116,2018-07-23 10:06:00,2018-08-09 00:00:00,2018-08-10 00:00:00,479046,TH,2013-06-06 10:04:00,3.0,Resort,Pay Later,...,0.0,1.0,0.0,0.0,0.0,2018-07-23,1506,False,False,616


In [1210]:

full_data.accommadation_type_name = full_data.accommadation_type_name.astype('category')
full_data.customer_nationality = full_data.customer_nationality.astype(str).astype('category')
#full_data.hotel_id = full_data.hotel_id.astype('category')
full_data.charge_option = full_data.charge_option.astype('category')
#full_data.booking_datetime = full_data.booking_datetime.astype('category')
full_data.cancellation_policy_code = full_data.cancellation_policy_code.astype('category')
full_data['customer_nationality_codes'] = full_data.customer_nationality.cat.codes
full_data['accommadation_type_name_codes'] = full_data.accommadation_type_name.cat.codes
#full_data['hotel_id_codes'] = full_data.hotel_id.cat.codes
full_data['charge_option_codes'] = full_data.charge_option.cat.codes
full_data['cancellation_policy_code_codes'] = full_data.cancellation_policy_code.cat.codes
b1 = full_data['booking_datetime'].aggregate(pd.Timestamp)
b2 = full_data['checkin_date'].aggregate(pd.Timestamp)
b2 = pd.to_datetime(full_data['checkin_date']).apply(lambda x: x.date())
b1 = pd.to_datetime(full_data['booking_datetime']).apply(lambda x: x.date())
# how many days before check-in the order was made
full_data['days_before_checkin'] = [i.days for i in (b2.to_numpy() - b1.to_numpy())]
full_data['days_before_checkin']


0        11
1         6
2         0
3         7
4        17
         ..
58654    17
58655     1
58656    62
58657     8
58658     5
Name: days_before_checkin, Length: 58659, dtype: int64

In [1211]:
(pd.Timestamp('2020-03-02')-pd.Timestamp('2020-02-01')).days

30

In [1212]:
dates = {"2018-07-17", "2018-07-18", "2018-07-19", "2018-07-20", "2018-07-21", "2018-07-22", "2018-07-23"}
end_cancellation_period = pd.Timestamp("2018-07-23")
# full_data['label'] = full_data.cancellation_datetime.map(lambda x: (end_cancellation_period - pd.Timestamp(x) ).days if x != 10000 else x)
# full_data['label'] = full_data['label'].map(lambda x: 10000 if x<0 else x)
full_data['label'] = full_data.cancellation_datetime.map(lambda x: bool(x) & True)
c1 = full_data['cancellation_datetime'].apply(lambda x: pd.Timestamp(x) if x != False else x)
c2 = full_data['booking_datetime'].apply(lambda x: pd.Timestamp(x))
# c3 = [i.days for i in (c2.to_numpy() - c1.to_numpy())]
# full_data['label'] = [i.days for i in (b2.to_numpy() - b1.to_numpy())]
# full_data['label'] = full_data.cancellation_datetime.map(lambda x: x > '2018-08-15' and x < '2018-08-20')
# c1 = full_data['cancellation_datetime'].apply(lambda x: (end_cancellation_period - pd.Timestamp(x) ).days if x != False else x)

# c1
# full_data['booking_datetime']

# how many days after booking, the order was cancelled
full_data['label'] = [abs((c1[i]-c2[i]).days)+1  if c1[i] != False else 0 for i in range(len(c2))]
full_data.loc[:, ['booking_datetime','cancellation_datetime', 'label']]



,booking_datetime,cancellation_datetime,label
0,2018-06-28 21:15:00,False,0
1,2018-08-10 22:31:00,False,0
2,2018-09-14 07:55:00,False,0
3,2018-06-25 07:33:00,False,0
4,2018-07-23 10:06:00,2018-07-23,2
...,...,...,...
58654,2018-07-18 22:07:00,2018-07-22,4
58655,2018-09-05 12:07:00,False,0
58656,2018-06-08 23:23:00,False,0
58657,2018-09-02 08:18:00,False,0


In [1213]:
# split the data into training set and test set
train_ratio = 0.75
rand_nums = np.random.random_sample(len(full_data))
full_data['train'] =  rand_nums <= train_ratio
full_data.train.value_counts()
train_df = full_data[full_data.train]
test_df = full_data[~full_data.train]
len(train_df), len(test_df), len(full_data)

(44087, 14572, 58659)

In [1214]:
features_cols = ["hotel_id", "days_before_checkin",
                "accommadation_type_name_codes",
                 "charge_option_codes",
                "hotel_star_rating",
                "customer_nationality_codes",
                 "cancellation_policy_code_codes"]
train_features = train_df[features_cols]
test_features = test_df[features_cols]
train_features.head()


,hotel_id,days_before_checkin,accommadation_type_name_codes,charge_option_codes,hotel_star_rating,customer_nationality_codes,cancellation_policy_code_codes
1,47729,6,10,0,4.0,114,152
2,780431,0,10,1,4.0,115,245
3,291365,7,10,0,3.0,110,151
6,400966,3,19,1,3.5,121,152
7,69287,23,10,0,3.5,120,152


In [1215]:
# features = full_data[[#"h_booking_id",
#                       "hotel_id",
#                       "accommadation_type_name_codes",
#                       "hotel_star_rating",
#                       "customer_nationality_codes"]]
# features.head()


In [1216]:
train_labels = train_df["label"]
test_labels = test_df["label"]
train_labels.head()

1     0
2     0
3     0
6     0
7    12
Name: label, dtype: int64

In [1217]:
random_classifier = RandomForestClassifier()
random_forest_regressor = RandomForestRegressor()

In [1218]:
y_train = train_df.label.values
X_train = train_features.values
y_test = test_df.label.values
X_test = test_features.values
X_train

array([[4.772900e+04, 6.000000e+00, 1.000000e+01, ..., 4.000000e+00,
        1.140000e+02, 1.520000e+02],
       [7.804310e+05, 0.000000e+00, 1.000000e+01, ..., 4.000000e+00,
        1.150000e+02, 2.450000e+02],
       [2.913650e+05, 7.000000e+00, 1.000000e+01, ..., 3.000000e+00,
        1.100000e+02, 1.510000e+02],
       ...,
       [4.071560e+05, 6.200000e+01, 1.000000e+01, ..., 5.000000e+00,
        2.500000e+01, 3.400000e+02],
       [3.031860e+05, 8.000000e+00, 9.000000e+00, ..., 0.000000e+00,
        8.200000e+01, 3.400000e+02],
       [2.227632e+06, 5.000000e+00, 1.000000e+01, ..., 4.000000e+00,
        2.500000e+01, 3.400000e+02]])

In [1219]:
model = random_classifier.fit(X_train,y_train)
predicted_cancellations = model.predict(X_test)
model2 = random_forest_regressor.fit(X_train,y_train)


In [1220]:
# how many orders got actually cancelled
y_test = np.array([False if i==0 else True for i in y_test])
np.bincount(y_test)

array([10656,  3916], dtype=int64)

In [1225]:
# classifier model prediction
predicted_cancellations = np.array([False if i==0 else True for i in predicted_cancellations])
np.bincount(predicted_cancellations & y_test)

array([13043,  1529], dtype=int64)

In [1232]:
# regression model prediction - how many days after booking was the order cancelled? 0 means it was not
reg_pred_cancellations = model2.predict(X_test)
reg_pred_cancellations  = np.array([False if i < 2 else True for i in reg_pred_cancellations])
np.bincount(reg_pred_cancellations)


array([9189, 5383], dtype=int64)

In [1228]:
# how many orders the model predicted were cancelled, on a data set he was trained on
# np.bincount(model.predict(X_train))


array([32469,  1399,  3740,   572,   459,   379,   328,   259,   240,
         190,   194,   187,   132,   151,   146,   138,   106,    84,
          87,    79,    92,    94,    88,    76,    68,    77,    54,
          75,    64,    58,    53,    52,    59,    55,    68,    53,
          35,    34,    48,    47,    33,    21,    34,    47,    34,
          33,    36,    29,    32,    29,    30,    15,    17,    21,
          19,    33,    33,    36,    11,    29,    18,    14,     9,
          18,    22,    11,    16,    13,    26,    17,    17,    11,
          21,    14,    12,    13,    17,    14,    11,    13,    11,
           9,    11,    17,     9,    12,    11,    15,    10,     6,
           9,    13,     8,     9,     8,     9,    13,    17,     7,
           6,     8,     5,     8,     5,     5,    12,     8,     6,
           6,    11,     8,     9,     5,     3,    10,     7,     1,
           4,     4,     6,     8,     3,     5,     3,     5,     0,
           4,     1,

In [ ]:
#np.bincount(y_train)

In [ ]:
#model.predict(X_test)

In [ ]:
# 0 means the order was not cancelled, i means the order got cancelled i days after it was booked
# np.bincount((predicted_cancellations & y_test)*test_df['cancelled_days_after_booking'])

# model.predict(X_test)
# model2.predict(X_test)

In [ ]:
#np.bincount(model.predict(X_train) & y_train)

In [ ]:
#full_data.label.value_counts()

In [ ]:
#full_data[(full_data.cancellation_datetime > '2018-08-07') & (full_data.cancellation_datetime < '2018-08-20')]

In [ ]:
test_data_w1 = pd.read_csv(r"C:\Users\97250\PycharmProjects\IML.HUJI\datasets\test_set_week_1.csv").dropna().drop_duplicates()
test_data_w1.head()

In [ ]:
# convert accomodation type name to codes